In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader , SubsetRandomSampler,Dataset
from torch.nn.utils.rnn import pad_packed_sequence,pack_padded_sequence,pad_sequence
from sklearn.metrics import classification_report, accuracy_score , f1_score

In [49]:
device = torch.device('cuda:7')
EMBED_DIM = 300
HIDDEN_DIM = 256
ATTENTION_DIM = 256

In [50]:
#Data Preprocessing for dataset1
positive = open("./data/data1/pos_train.txt","r").read()
negative = open("./data/data1/neg_train.txt","r").read()
val_positive = open("./data/data1/pos_valid.txt","r").read()
val_negative = open("./data/data1/neg_valid.txt","r").read()

pos_set = positive.split("\n")[:-1]
neg_set = negative.split("\n")[:-1]
val_pos_set = val_positive.split("\n")[:-1]
val_neg_set = val_negative.split("\n")[:-1]

In [51]:
vocabulary = set()
word_to_idx = {}
idx = 1
for sen in pos_set:
    words = sen.split(" ")
    for word in words:
        if word not in word_to_idx.keys():
            word_to_idx[word] = idx 
            idx = idx + 1
        vocabulary.add(word)
for sen in neg_set:
    words = sen.split(" ")
    for word in words:
        if word not in word_to_idx.keys():
            word_to_idx[word] = idx 
            idx = idx + 1
        vocabulary.add(word)
for sen in val_pos_set:
    words = sen.split(" ")
    for word in words:
        if word not in word_to_idx.keys():
            word_to_idx[word] = idx 
            idx = idx + 1
        vocabulary.add(word)
for sen in val_neg_set:
    words = sen.split(" ")
    for word in words:
        if word not in word_to_idx.keys():
            word_to_idx[word] = idx 
            idx = idx + 1
        vocabulary.add(word)
print(len(vocabulary))

12542


In [52]:
no_of_sentences = 40
max_sen_len = 10

In [74]:
dataset = []
for sen in pos_set:
    words = sen.split(" ")
    src = [word_to_idx[word] for word in words]
    # while(len(src)<400):
    #     src.append(0)
    dataset.append((src,0))
for sen in neg_set:
    words = sen.split(" ")
    src = [word_to_idx[word] for word in words]
    # while(len(src)<400):
    #     src.append(0)
    dataset.append((src,1))
print(len(dataset))
for sen in val_pos_set:
    words = sen.split(" ")
    src = [word_to_idx[word] for word in words]
    # while(len(src)<400):
    #     src.append(0)
    dataset.append((src,0))
for sen in val_neg_set:
    words = sen.split(" ")
    src = [word_to_idx[word] for word in words]
    # while(len(src)<400):
    #     src.append(0)
    dataset.append((src,1))
print(len(dataset))



12664
17998


In [75]:
glove_vectors = GloVe(name='840B',dim=300)

embeddings = torch.zeros(len(word_to_idx)+1,300)
for word , id in word_to_idx.items():
    embeddings[id] = glove_vectors[word]

In [76]:
def getEmbedding(sen):
    word = []
    length = 0
    for i in sen:
        word.append(embeddings[i])
        length+=1
    return torch.stack(word) , length


In [77]:
# def preprocessData(sentence):
#     sentences = [sentence[i:i+10] for i in range(0,400,10)]
#     sentences_lengths=[]
#     num_sentences = 0
#     review_tokens=[]
#     for sentence in sentences:
#         embed , length = getEmbedding(sentence)
#         review_tokens.append(embed)
#         sentences_lengths.append(length)
#         num_sentences +=1
#     return torch.nn.utils.rnn.pad_sequence(review_tokens,batch_first=True),sentences_lengths,num_sentences

def preprocessData(sentence):
    sentences_lengths=[]
    num_sentences = 0
    review_tokens=[]
    embed , length = getEmbedding(sentence)
    review_tokens.append(embed)
    sentences_lengths.append(length)
    num_sentences +=1
    return torch.nn.utils.rnn.pad_sequence(review_tokens,batch_first=True),sentences_lengths,num_sentences

    

In [78]:
processed_dataset=[]
print(5)
for review in dataset:
    tokens , sent_len , num_sent = preprocessData(review[0])
    processed_dataset.append({'review': tokens,'sent_lengths': sent_len,'length' : num_sent,'label' : review[1]})

5


In [79]:
class ReviewDataSet(Dataset):
    
    def __init__(self,reviews):
        super().__init__()
        self.reviews = reviews
        
    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        return self.reviews[index]

In [80]:
def collate_function(batch_data):   
    
    inputs = [b['review'] for b in batch_data]
    sent_lengths = [ b['sent_lengths'] for b in batch_data ]
    n_sentences = [ b['length'] for b in batch_data ]
    labels = torch.tensor([b['label'] for b in batch_data])


    labels = labels.unsqueeze(1)
    max_n_sentences = max([i.shape[0] for i in inputs] )
    max_n_words = max([i.shape[1] for i in inputs])

 
    processed_inputs = []
    for inp in inputs:

        t1 = torch.permute(inp,(2,1,0))
        t1 = torch.nn.functional.pad(t1,(0,max_n_sentences-inp.shape[0],0,max_n_words-inp.shape[1]))
        t1 = torch.permute(t1,(2,1,0))
        processed_inputs.append(t1)

    final_inp = torch.stack(processed_inputs)
    #inputs = pad_sequence(inputs,batch_first=True)
    return  {'input' : final_inp , 'sent_lengths': sent_lengths , 'lengths' : n_sentences ,'labels' : labels }

In [81]:
# Train-Valid split of 80-20
def split(n, ratio):
    idxs = np.random.permutation(n)
    return idxs[:ratio], idxs[ratio:]

train_indices, val_indices = split(len(dataset), 12542)

dataset = ReviewDataSet(processed_dataset)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_dataloader = DataLoader(dataset,64,sampler=train_sampler,collate_fn=collate_function)
valid_dataloader = DataLoader(dataset,64,sampler=valid_sampler,collate_fn=collate_function)


In [82]:
batch_data = next(iter(train_dataloader))

In [83]:
#Word Level Attention
class wordAttention(nn.Module):
    
    def __init__(self, embed_size,word_gru_hidden, bidirectional= True,dropout=0.3):        
        
        super(wordAttention, self).__init__()
        
        # self.num_tokens = num_tokens
        self.embed_size = embed_size
        self.word_gru_hidden = word_gru_hidden
        self.bidirectional = bidirectional
        if bidirectional == True:
            self.word_gru = nn.GRU(embed_size, word_gru_hidden, bidirectional= True)
            self.weight_W_word = nn.Linear(2* word_gru_hidden,2*word_gru_hidden)
            self.u_w_word = nn.Linear(2*word_gru_hidden, 1)
        else:
            self.word_gru = nn.GRU(embed_size, word_gru_hidden, bidirectional= False)
            self.weight_W_word = nn.Linear(word_gru_hidden,word_gru_hidden)
            self.u_w_word = nn.Linear(word_gru_hidden, 1)
            
        self.dropout = nn.Dropout(dropout)

        
        
    def forward(self, input, inp_len):
        # word level gru
        packed_embedding = pack_padded_sequence(input,inp_len,batch_first=True)
        outputs,hidden = self.word_gru(packed_embedding)
        outputs,_ =pad_packed_sequence(outputs,batch_first=True)
        

        attention_outs = torch.tanh(self.dropout(self.weight_W_word(outputs)))
        attention_scores = self.u_w_word(attention_outs)
        attention_scores = attention_scores.squeeze(2)
        attention_probs = F.softmax(attention_scores,dim=1)
        attention_probs = attention_probs.unsqueeze(2)

        weighted_embeddings = attention_probs * outputs
        outputs = torch.sum(weighted_embeddings,dim=1)
        return outputs

In [84]:

#Sentence Level Attention

class sentAttention(nn.Module):
    
    
    def __init__(self,word_gru_hidden, sent_gru_hidden, bidirectional= True , dropout=0.3):        
        
        super(sentAttention, self).__init__()
        
        self.sent_gru_hidden = sent_gru_hidden
        self.bidirectional = bidirectional
        
        
        if bidirectional == True:
            self.sent_gru = nn.GRU(2*word_gru_hidden, sent_gru_hidden, bidirectional= True)        
            self.weight_W_sent = nn.Linear(2* sent_gru_hidden,2*sent_gru_hidden)
            self.u_w_sent = nn.Linear(2*sent_gru_hidden, 1)
        else:
            self.sent_gru = nn.GRU(word_gru_hidden, sent_gru_hidden, bidirectional= False)        
            self.weight_W_sent = nn.Linear(sent_gru_hidden,2*sent_gru_hidden)
            self.u_w_sent = nn.Linear(sent_gru_hidden, 1)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, sent, sent_len):
        # sentence level gru
        packed_embedding = pack_padded_sequence(sent,sent_len,enforce_sorted=False)
        outputs,hidden = self.sent_gru(packed_embedding)
        outputs,_ = pad_packed_sequence(outputs,batch_first=True)

        attention_outs = torch.tanh(self.dropout(self.weight_W_sent(outputs)))
        attention_scores = self.u_w_sent(attention_outs)
        attention_scores = attention_scores.squeeze(2)
        attention_probs = F.softmax(attention_scores,dim=1)
        attention_probs = attention_probs.unsqueeze(2)

        weighted_embeddings = attention_probs * outputs
        outputs = torch.sum(weighted_embeddings,dim=1)
        return outputs

In [85]:

#Heirarichial Attention

class HAN(nn.Module):
    def __init__(self,embed_size,word_gru_hidden ,sent_gru_hidden, n_classes ,dropout=0.3,bidirectional=True):
        super(HAN, self).__init__()

        self.embed_size = embed_size
        self.word_gru_hidden = word_gru_hidden
        self.dropout=dropout
        self.bidirectional = bidirectional
        # self.sent_gru_hidden = 2*word_gru_hidden if bidirectional else word_gru_hidden
        self.sent_gru_hidden = sent_gru_hidden

        self.word_attention = wordAttention(embed_size , word_gru_hidden , bidirectional , dropout)
        self.sent_attention = sentAttention(word_gru_hidden, self.sent_gru_hidden, bidirectional, dropout)
        self.fc1 = nn.Linear(2*self.sent_gru_hidden ,self.word_gru_hidden)
        self.fc2 = nn.Linear(self.word_gru_hidden , self.word_gru_hidden>>1)
        self.final = nn.Linear(word_gru_hidden>>1 , n_classes)
    
    def forward(self,inputs,inp_sen_len , inp_word_len):
        sentence_embeddings =[]
        for i in range(inputs.shape[0]):
            sentence_embeddings.append(self.word_attention(inputs[i] , inp_word_len[i]))
        batch_sentences = pad_sequence(sentence_embeddings)

        doc_embedding = self.sent_attention(batch_sentences,inp_sen_len)
        output = self.fc1(doc_embedding)
        output = self.fc2(output)
        output = self.final(output)
        output = F.softmax(output,dim=1)

        return output

In [86]:
HAN_Model = HAN(embed_size=300 , word_gru_hidden=128 ,sent_gru_hidden=128,n_classes=2)
# HAN_Model(batch_data['input'] , batch_data['lengths'] , batch_data['sent_lengths'])

In [87]:
HAN_Model.to(device)

HAN(
  (word_attention): wordAttention(
    (word_gru): GRU(300, 128, bidirectional=True)
    (weight_W_word): Linear(in_features=256, out_features=256, bias=True)
    (u_w_word): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (sent_attention): sentAttention(
    (sent_gru): GRU(256, 128, bidirectional=True)
    (weight_W_sent): Linear(in_features=256, out_features=256, bias=True)
    (u_w_sent): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (final): Linear(in_features=64, out_features=2, bias=True)
)

In [88]:
optimizer = torch.optim.Adam(HAN_Model.parameters() , lr = 1e-3)

In [89]:
from tqdm import tqdm
num_epochs = 20
best_val_loss = 100
best_accuracy =0
for epoch in range(1,num_epochs+1):
    epoch_loss =0
    HAN_Model.train()
    train_labels = []
    train_pred =[]
    correct_train , B_train =0 ,0
    #Training
    for data in tqdm(train_dataloader):
        optimizer.zero_grad()

        input , lengths , sent_len , labels = data['input'] , data['lengths'] , data['sent_lengths'] , data['labels']
        input = input.to(device)
        # lengths = lengths.to(device)
        # sent_len = sent_len.to(device)
        labels = labels.to(device)

        y_pred = HAN_Model(input,lengths,sent_len)
        loss = F.cross_entropy(y_pred , labels.squeeze(1))

        loss.backward()
        optimizer.step()
        epoch_loss += float(loss)

        y_pred = torch.softmax(y_pred , dim=1).argmax(dim=1)


        train_labels.extend(labels.cpu().detach().numpy())
        train_pred.extend(y_pred.cpu().detach().numpy())

    print(f'Epoch : {epoch} Training Loss : {epoch_loss/len(train_dataloader)} Train Accuracy : {accuracy_score(train_labels ,train_pred)*100} ')

    #Validating
    valid_loss=0
    valid_labels = []
    valid_pred =[]
    for data in tqdm(valid_dataloader):

        input , lengths , sent_len , labels = data['input'] , data['lengths'] , data['sent_lengths'] , data['labels']
        input = input.to(device)
        # lengths = lengths.to(device)
        # sent_len = sent_len.to(device)
        labels = labels.to(device)

        y_pred = HAN_Model(input,lengths,sent_len)
        loss = F.cross_entropy(y_pred , labels.squeeze(1))

        valid_loss += float(loss)

        y_pred = torch.softmax(y_pred , dim=1).argmax(dim=1)

        valid_labels.extend(labels.cpu().detach().numpy())
        valid_pred.extend(y_pred.cpu().detach().numpy())
    valid_loss = valid_loss/len(valid_dataloader)
    accuracy = round(accuracy_score(valid_labels ,valid_pred)*100,4)
    print(f'Validation Loss : {valid_loss} Validation Accuracy : {accuracy} ')
    if(accuracy > best_accuracy):
        i=epoch
        best_val_loss = valid_loss
        best_accuracy = accuracy
        torch.save(HAN_Model.state_dict() , "Best Model.pt")

print("\nTraining Completed")

print(f"\n Best Epoch : {i} Valid Loss : {best_val_loss} Accuracy : {best_accuracy}")




        

100%|██████████| 196/196 [00:27<00:00,  7.22it/s]


Epoch : 1 Training Loss : 0.40674157045325454 Train Accuracy : 90.17700526231862 


100%|██████████| 86/86 [00:06<00:00, 14.33it/s]


Validation Loss : 0.38664953133394553 Validation Accuracy : 92.5403 


100%|██████████| 196/196 [00:26<00:00,  7.35it/s]


...

Validation Loss : 0.364372523718102 Validation Accuracy : 94.813 


100%|██████████| 196/196 [00:27<00:00,  7.02it/s]


Epoch : 20 Training Loss : 0.34826517515644734 Train Accuracy : 96.43597512358475 


100%|██████████| 86/86 [00:06<00:00, 13.27it/s]

Validation Loss : 0.364518245638803 Validation Accuracy : 94.8497 

Training Completed

 Best Epoch : 16 Valid Loss : 0.36294150768324385 Accuracy : 94.9047


In [92]:
#Dataset 2
valid_HAN = HAN(embed_size=300 , word_gru_hidden=128 ,sent_gru_hidden=128,n_classes=2)
valid_HAN.to(device)
valid_HAN.load_state_dict(torch.load("/data5/home/bhargavav2/DLNLP/Assignment2/PartB/Best Model1.pt"))
valid_HAN.eval()

HAN(
  (word_attention): wordAttention(
    (word_gru): GRU(300, 128, bidirectional=True)
    (weight_W_word): Linear(in_features=256, out_features=256, bias=True)
    (u_w_word): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (sent_attention): sentAttention(
    (sent_gru): GRU(256, 128, bidirectional=True)
    (weight_W_sent): Linear(in_features=256, out_features=256, bias=True)
    (u_w_sent): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (final): Linear(in_features=64, out_features=2, bias=True)
)

In [94]:
#Validation
from tqdm import tqdm
valid_loss=0
valid_labels = []
valid_pred =[]
for data in tqdm(valid_dataloader):

    input , lengths , sent_len , labels = data['input'] , data['lengths'] , data['sent_lengths'] , data['labels']
    input = input.to(device)
    labels = labels.to(device)

    y_pred = valid_HAN(input,lengths,sent_len)
    loss = F.cross_entropy(y_pred , labels.squeeze(1))

    valid_loss += float(loss)

    y_pred = torch.softmax(y_pred , dim=1).argmax(dim=1)

    valid_labels.extend(labels.cpu().detach().numpy())
    valid_pred.extend(y_pred.cpu().detach().numpy())
valid_loss = valid_loss/len(valid_dataloader)
accuracy = round(accuracy_score(valid_labels ,valid_pred)*100,4)
valid_labels = [i[0] for i in valid_labels]
print(valid_labels)
print(valid_pred)
print(f'Validation Loss : {valid_loss}    Validation Accuracy : {accuracy}    f1-score : {round(f1_score(valid_labels, valid_pred, average="micro"),4)}')

100%|██████████| 86/86 [00:06<00:00, 13.82it/s]

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 